[x] Make NER microservice.

[x] Make server_crypto.py and server_finance.py.

[x] Make SA microservice.

[x] Make server_crypto.py and server_finance.py.

[x] Get Free Tier MongoDB Atlas.

[x] NER from entities collection.

[ ] Make script to download daily crypto chit-chat data.

References:
https://arxiv.org/pdf/1905.03423.pdf

In [1]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import grpc
import ner_pb2
import ner_pb2_grpc
if "../../ner" not in sys.path:
    sys.path.append("../../ner")

In [2]:
%load_ext autoreload
%autoreload 2
from server import NER
ner = NER()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
ner_request =     ner_pb2.NerRequest(
        text="I love Bitcoin stocks and ETH.",
        metadata="{\"request\":{\"domains\":[\"crypto\"]}}"
    )
ner.ExtractEntities(request=ner_request,context = None)

text: "I love Bitcoin stocks and ETH."
metadata: "{\"request\": {\"domains\": [\"crypto\"]}, \"ner\": {\"ner_version\": \"naivekw_0\"}}"
results: "{\"BTC\": {\"span\": [7, 14], \"match\": \"Bitcoin\", \"domain\": \"crypto\"}, \"ETH\": {\"span\": [26, 29], \"match\": \"ETH\", \"domain\": \"crypto\"}}"

In [4]:


ner_channel = grpc.insecure_channel(
    os.environ.get("NER_SERVER_ADDRESS")
)
ner_stub = ner_pb2_grpc.NERStub(ner_channel)

ner_response = ner_stub.ExtractEntities(
    ner_pb2.NerRequest(
        text="I love Stacks stocks.",
        metadata="kek"
    )
)

ner_response

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: Expecting value: line 1 column 1 (char 0)"
	debug_error_string = "{"created":"@1645138506.581895559","description":"Error received from peer ipv4:172.24.0.8:50052","file":"src/core/lib/surface/call.cc","file_line":1074,"grpc_message":"Exception calling application: Expecting value: line 1 column 1 (char 0)","grpc_status":2}"
>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [1]:
r = requests.get("https://coinmarketcap.com/all/views/all/")
soup = BeautifulSoup(r.content,'html.parser')
data = []

for row in soup.find_all('tr'):
    row_data = []
    for item in row.find_all('td')[:-1]:
        row_data.append(item.text)
    data.append(row_data)
data_cryptos = pd.DataFrame(data, columns = ["Rank","Name","Symbol","Market Cap","Price","Circulating Supply", "Volumne(24h)","%1h","%24h","%7d"])
data_cryptos = data_cryptos[["Name","Symbol"]].dropna()

In [2]:
crypto_entities = []
for _,row in data_cryptos.iterrows():
    crypto_entities.append({
        "entity":row["Symbol"],
        "domain":"crypto",
        "synonims":[row["Symbol"],row["Name"][len(row["Symbol"]):]]
    })
crypto_entities

[{'entity': 'BTC', 'domain': 'crypto', 'synonims': ['BTC', 'Bitcoin']},
 {'entity': 'ETH', 'domain': 'crypto', 'synonims': ['ETH', 'Ethereum']},
 {'entity': 'USDT', 'domain': 'crypto', 'synonims': ['USDT', 'Tether']},
 {'entity': 'BNB', 'domain': 'crypto', 'synonims': ['BNB', 'BNB']},
 {'entity': 'USDC', 'domain': 'crypto', 'synonims': ['USDC', 'USD Coin']},
 {'entity': 'XRP', 'domain': 'crypto', 'synonims': ['XRP', 'XRP']},
 {'entity': 'ADA', 'domain': 'crypto', 'synonims': ['ADA', 'Cardano']},
 {'entity': 'SOL', 'domain': 'crypto', 'synonims': ['SOL', 'Solana']},
 {'entity': 'LUNA', 'domain': 'crypto', 'synonims': ['LUNA', 'Terra']},
 {'entity': 'DOT', 'domain': 'crypto', 'synonims': ['DOT', 'Polkadot']},
 {'entity': 'AVAX', 'domain': 'crypto', 'synonims': ['AVAX', 'Avalanche']},
 {'entity': 'DOGE', 'domain': 'crypto', 'synonims': ['DOGE', 'Dogecoin']},
 {'entity': 'SHIB', 'domain': 'crypto', 'synonims': ['SHIB', 'Shiba Inu']},
 {'entity': 'BUSD', 'domain': 'crypto', 'synonims': ['BU

In [3]:
import os
from pymongo import MongoClient
client = MongoClient(os.getenv("MONGO_CONNECTION_STRING"))
db = client[os.getenv("MONGO_DB_NAME")]
#db.entities.insert_many(crypto_entities)

In [11]:
sp500 = pd.read_csv("../../ner/s&p.csv")
sp_entities = []
for _, row in sp500.iterrows():
    sp_entities.append({
        "entity":row["Symbol"],
        "domain":"finance",
        "sector":row["Sector"],
        "synonims":[row["Symbol"],row["Name"]]
    })
db.entities.insert_many(sp_entities)

In [8]:
import os
import sys
if ".../../ner" not in sys.path:
    sys.path.append("../../ner")

from server_crypto import NER
ner_stub = NER()

{'BTC': {'synonims': ['BTC', 'Bitcoin'], 'domain': 'crypto'}, 'ETH': {'synonims': ['ETH', 'Ethereum'], 'domain': 'crypto'}, 'USDT': {'synonims': ['USDT', 'Tether'], 'domain': 'crypto'}, 'BNB': {'synonims': ['BNB', 'BNB'], 'domain': 'crypto'}, 'USDC': {'synonims': ['USDC', 'USD Coin'], 'domain': 'crypto'}, 'XRP': {'synonims': ['XRP', 'XRP'], 'domain': 'crypto'}, 'ADA': {'synonims': ['ADA', 'Cardano'], 'domain': 'crypto'}, 'SOL': {'synonims': ['SOL', 'Solana'], 'domain': 'crypto'}, 'LUNA': {'synonims': ['LUNA', 'Terra'], 'domain': 'crypto'}, 'DOT': {'synonims': ['DOT', 'Polkadot'], 'domain': 'crypto'}, 'AVAX': {'synonims': ['AVAX', 'Avalanche'], 'domain': 'crypto'}, 'DOGE': {'synonims': ['DOGE', 'Dogecoin'], 'domain': 'crypto'}, 'SHIB': {'synonims': ['SHIB', 'Shiba Inu'], 'domain': 'crypto'}, 'BUSD': {'synonims': ['BUSD', 'Binance USD'], 'domain': 'crypto'}, 'MATIC': {'synonims': ['MATIC', 'Polygon'], 'domain': 'crypto'}, 'CRO': {'synonims': ['CRO', 'Crypto.com Coin'], 'domain': 'crypto'

In [1]:
ner_response = ner_stub.ExtractEntities(
    ner_pb2.NerRequest(
        text="I love btc stocks.",
        metadata="kek"
    ),
    context=None
)

ner_response

NameError: name 'ner_stub' is not defined

In [ ]:
r = requests.get("https://www.gkgigs.com/list-of-top-cryptocurrencies/#List_Of_Top_1000_Cryptocurrencies_In_The_World")
soup = BeautifulSoup(r.content,'html.parser')


In [19]:
data = []
for row in soup.find_all('tr')[22:]:
    row_data = []
    for item in row.find_all('td')[1:]:
        row_data.append(item.text)
    data.append(row_data)

In [23]:
data = data[1:-1]

In [29]:
docs = []
for d in data:
    if d:
        docs.append({
            "entity":d[1],
            "synonims":d,
            "domain":"crypto"
        })

In [32]:
db.entities.delete_many({"domain":"crypto"})